In [33]:
import os, random
from dotenv import load_dotenv

import openai
import pandas as pd

In [34]:
load_dotenv()

API_KEY = os.getenv('API_KEY')
openai.api_key = API_KEY

In [35]:
DATA_PATH = './data/100886_from_300000_seq_len_from_500_to_1000'

file_names = os.listdir(DATA_PATH)
file_names = [file_name for file_name in file_names if 'labeled' not in file_name]
file_names = sorted(file_names, key=lambda x: int(x.split('.')[0].split('_')[-1]))
file_names = [os.path.join(DATA_PATH, file_name) for file_name in file_names]

In [36]:
# df = pd.read_pickle(file_names[0]).reset_index(drop=True)
# df['label'] = 'to be labeled'

In [37]:
role = 'you are a data labeller who finds key-phrases in a news article.'
prompt_starter = "find top 10 most important key-phrases in the article and separate the key-phrases with semi-colons; numbering is not needed; don't start nor end it with any punctuation:\n\n"

In [38]:
def generate_label(role, prompt):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": role},
            {"role": "user", "content": prompt},
    ]
    )
  
    response_content = response['choices'][0]['message']['content']
    return response_content

In [39]:
### Define Range

start = 0
end = 1

In [42]:
for file_name in file_names[start:end]:
    df = pd.read_pickle(file_name).reset_index(drop=True)
    df['label'] = 'to be labeled'

    for idx, row in df.iterrows():
        # if idx == 1:
        #     break
        prompt = prompt_starter + row.title_content
        label = generate_label(role, prompt)
        df.loc[idx, 'label'] = label

    save_name = file_name.split('.pickle')[0] + '_labeled.pickle'
    df.to_pickle(save_name)

### Test Generation

In [ ]:
# idx = random.randint(0, len(df))
# sample = df.iloc[idx]
# article = sample.title_content
# sample, article

In [ ]:
# role = 'you are a data labeller who finds key-phrases in a news article.'
# prompt_starter = "find top 10 most important key-phrases in the article and separate the key-phrases with semi-colons; numbering is not needed; don't start nor end it with any punctuation:\n\n"

# prompt = prompt_starter + article
# print(prompt)

In [ ]:
# ### prompt engineering v1

# response = openai.ChatCompletion.create(
#   model="gpt-3.5-turbo",
#   messages=[
#         {"role": "system", "content": role},
#         {"role": "user", "content": prompt},
#     ]
# )

# response['choices'][0]['message']['content']